### Implementing with respect to GRU RNN
Everything else is the same. Only LSTM RNN is replaced with GRU RNN

In [1]:
## Data collection
import nltk 
nltk.download('gutenberg')
from nltk.corpus import gutenberg 
import pandas as pd


## Load the dataset
data = gutenberg.raw('shakespeare-hamlet.txt')

## Save to a file
with open('hamlet.txt', 'w') as file:
    file.write(data)

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\fmssh\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [2]:
## Data preprocessing
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

## Load the dataset and apply padding sequences and tokenizer

with open('hamlet.txt', 'r') as file:
    text = file.read().lower()

## Tokenize the text

tokenizer = Tokenizer()  ## Creating indexes for the words
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1
total_words

4818

In [3]:
## Creating input sequences

input_sequences =[]
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [4]:
## Apply pad sequences
max_sequence_len=max([len(x) for x in input_sequences])
max_sequence_len

14

In [5]:
## Converting into an array
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
input_sequences

array([[   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       [   0,    0,    0, ...,  687,    4,   45],
       ...,
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4],
       [   0,    0,    0, ..., 1047,    4,  193]], dtype=int32)

In [6]:
## Create predictors and label

import tensorflow as tf
x,y =input_sequences[:,:-1], input_sequences[:,-1]

In [7]:
## Creating categorical features
y = tf.keras.utils.to_categorical(y, num_classes=total_words)
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [8]:
## Splitting the dataset into train and test sets
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2)

In [10]:
## GRU RNN

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout

## Define the model
model = Sequential()
model.add(Embedding(total_words,100))
model.add(GRU(150,return_sequences=True))   ## For GRU, simply replace LSTM with GRU
model.add(Dropout(0.2))
model.add(GRU(100))   ## Same here
model.add(Dense(total_words,activation="softmax"))

## Build the model
model.build(input_shape=(None, max_sequence_len))

## Compile the model 
model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 14, 100)        │       481,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ (None, 14, 150)        │       113,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 14, 150)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ (None, 100)            │        75,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4818)           │       486,618 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,157,418 (4.42 MB)

 Trainable params: 1,157,418 (4.42 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
## Importing EarlyStopping callback to monitor validation loss
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', patience =5, restore_best_weights = True)


In [12]:
## Training the model 
history = model.fit(x_train, y_train, epochs=100, validation_data=(x_test, y_test), verbose=1)

Epoch 1/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - accuracy: 0.0288 - loss: 7.2053 - val_accuracy: 0.0305 - val_loss: 6.8212
Epoch 2/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.0350 - loss: 6.5229 - val_accuracy: 0.0466 - val_loss: 6.8431
Epoch 3/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.0530 - loss: 6.2468 - val_accuracy: 0.0581 - val_loss: 6.8613
Epoch 4/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.0662 - loss: 6.0266 - val_accuracy: 0.0709 - val_loss: 6.7872
Epoch 5/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.0778 - loss: 5.7242 - val_accuracy: 0.0731 - val_loss: 6.8970
Epoch 6/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.0920 - loss: 5.4382 - val_accuracy: 0.0711 - val_loss: 6.9651
Epoch 7/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.1011 - loss: 5.2039 - val_accuracy: 0.0711 - val_loss: 7.0772
Epoch 8/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.1156 - loss: 4.9403 -

In [13]:
## Function to predict the next word
def predict_next(model, tokenizer, text, max_sequence_len):
    token_list = tokenizer.texts_to_sequences([text])[0]
    if len(token_list) >= max_sequence_len:
        token_list = token_list[-(max_sequence_len-1):]  ## Ensure sequence length matches max_sequence_len -1
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict(token_list, verbose=0)
    predicted_word_index = np.argmax(predicted, axis=1)
    for word, index in tokenizer.word_index.items():
        if index == predicted_word_index:
            return word
    return None 

In [14]:
## Trying an input to see how the model performs in predicting next word
input_text = "To be or not to be"
print(f"Input Text: {input_text} ")
max_sequence_len=model.input_shape[1]+1
next_word = predict_next(model, tokenizer, input_text, max_sequence_len)
print(f"Next Word Prediction: {next_word}")

Input Text: To be or not to be 
Next Word Prediction: that


In [15]:
## Saving the model
model.save("Next_Word_GRU.h5")

## Saving the tokenizer as a .pkl file

import pickle
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)  

In [16]:
## Trying another input 

input_text = "To offer it the shew of"  ## input taken from dataset, line: 196
print(f"Input Text: {input_text} ")
max_sequence_len=model.input_shape[1]+1
next_word = predict_next(model, tokenizer, input_text, max_sequence_len)
print(f"Next Word Prediction: {next_word}")

Input Text: To offer it the shew of 
Next Word Prediction: violence


In [17]:
## Trying another input 
input_text = "For it is as the Ayre,"  ## input taken from dataset, line: 197
print(f"Input Text: {input_text} ")
max_sequence_len=model.input_shape[1]+1
next_word = predict_next(model, tokenizer, input_text, max_sequence_len)
print(f"Next Word Prediction: {next_word}")

Input Text: For it is as the Ayre, 
Next Word Prediction: invulnerable
